## Lambda Trajectory Analysis

### Step 1: Importing Libraries

In [ ]:
import numpy
import matplotlib.pyplot as plt
import glob
import pandas as pd

### Step 2: Analyse Protonation State Evolution Over Time for single .xvg file

In [ ]:
# Initialize lists to store time and protonation coordinate values
time = []
protonation_states = []

filename = 'cphmd-coord-1.xvg'

# Open and read the .xvg file line by line
with open(filename, 'r') as f:
    for line in f:
        # Skip comment and metadata lines that start with '#' or '@'
        if line.startswith(('#', '@')):
            continue
        # Split line into columns (assumes two columns: time and protonation coordinate)
        values = line.strip().split()
        time_ps = float(values[0])             
        time_ns = time_ps / 1000              
        time.append(time_ns)                   
        protonation_states.append(float(values[1]))  # Store lambda coordinate 

# Plotting the lambda coordinate over time 

plt.plot(time, protonation_states)
plt.xlabel("Time (ns)", fontsize=14)
plt.ylabel("Lambda Coordinate", fontsize=14)
plt.title("Lambda Evolution Over Time", fontsize=16)
plt.legend(ncol=3, fontsize=10, loc='upper right')
plt.grid(True, linestyle="--", alpha=0.6)
plt.tight_layout()
plt.savefig("lambda_evolution.png", dpi=300)
plt.show()

### Step 3: Analyse Running Average for lambda coordinates over time

In [ ]:
# Define running average window size (number of frames)
window_size = 10000

# Find all .xvg files in the current directory
xvg_files = glob.glob("*.xvg")

# Loop through each .xvg file and process
for filename in xvg_files:
    time = []
    lambda_values = []

    # Read the .xvg file line by line
    with open(filename, 'r') as f:
        for line in f:
            # Skip comments and metadata lines starting with '#' or '@'
            if line.startswith(('#', '@')):
                continue
            # Parse data values (assumes two columns: time and lambda)
            values = line.strip().split()
            time_ps = float(values[0])  
            time_ns = time_ps / 1000    
            time.append(time_ns)        
            lambda_values.append(float(values[1]))  # Lambda coordinate value

    # Convert collected data to a pandas DataFrame
    df = pd.DataFrame({'time': time, 'lambda': lambda_values})
    
    # Compute running average (rolling mean) with specified window size
    # This smooths the lambda coordinate over the window frames
    df['lambda_smoothed'] = df['lambda'].rolling(window=window_size).mean()

    # Drop initial NaN values generated by rolling window calculation
    df = df.dropna()

    # Plot raw and smoothed lambda values vs. time (in nanoseconds)
    plt.figure(figsize=(8, 5))
    plt.plot(df['time'], df['lambda'], label='Raw', alpha=0.4)
    plt.plot(df['time'], df['lambda_smoothed'], label=f'Running Avg (window={window_size})', linewidth=2, color='orange')

    plt.xlabel("Time (ns)", fontsize=14)
    plt.ylabel("Lambda Coordinate", fontsize=14)
    plt.title(f"Lambda Evolution Over Time ({filename})", fontsize=16)
    plt.legend(fontsize=10, loc='upper left', bbox_to_anchor=(0.5, -0.15), ncol=2, frameon=False)
    plt.grid(True, linestyle="--", alpha=0.6)
    plt.tight_layout()
    plt.subplots_adjust(bottom=0.25)  

    # Save the plot to PNG file 
    output_filename = filename.replace(".xvg", "_lambda_smoothed.png")
    plt.savefig(output_filename, dpi=300, bbox_inches='tight')
    plt.close()